In [1]:
# import data
import pandas as pd
import numpy as np

train = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/ANN/data/titanic/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/ANN/data/titanic/test.csv')
gender_submission = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/ANN/data/titanic/gender_submission.csv')

train.shape, test.shape, gender_submission.shape

((891, 12), (418, 11), (418, 2))

In [2]:
# find the target column
train_cols = train.columns
test_cols = test.columns

for col in train_cols:
  if col not in test_cols:
    print(col)

Survived


In [3]:
# drop features
train.drop(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], axis=1, inplace=True)
test.drop(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], axis=1, inplace=True)

# fill na
train.fillna(train.mean(), inplace=True)
test.fillna(test.mean(), inplace=True)

In [4]:
# create X and y
X = train.drop('Survived', axis=1)
y = train['Survived']

In [5]:
# scale data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_test = scaler.transform(test)

X.shape, y.shape, X_test.shape

((891, 6), (891,), (418, 6))

# Building the network

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Titanic(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(6, 20)
    self.fc2 = nn.Linear(20, 20)
    self.output = nn.Linear(20, 1)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = torch.sigmoid(self.output(x))
    return x

In [7]:
model = Titanic()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 50000
for epoch in range(epochs):
  inputs = torch.tensor(X, dtype=torch.float32)
  labels = torch.tensor(y, dtype=torch.float32)

  # forward pass
  outputs = model(inputs)
  loss = criterion(outputs, labels.unsqueeze(1))

  # backward pass and optimization
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  if (epoch+1) % 10000 == 0:
    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}')

Epoch 10000/50000, Loss: 0.1702
Epoch 20000/50000, Loss: 0.1391
Epoch 30000/50000, Loss: 0.1271
Epoch 40000/50000, Loss: 0.1156
Epoch 50000/50000, Loss: 0.1033


In [8]:
with torch.no_grad():
  y_predicted = model(torch.tensor(X_test, dtype=torch.float32))
  y_predicted_cls = y_predicted.round()

In [9]:
prediction_numpy = y_predicted_cls.squeeze(1).numpy().astype(int)
gender_submission['Survived'] = prediction_numpy
gender_submission.to_csv('submission.csv', index=False)